In [9]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [11]:
%run -i ../../zkstats/core.py

In [12]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [13]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.sqrt(torch.var(dummy_data_tensor, correction = 1))
dummy_data_mean = torch.mean(dummy_data_tensor)

In [14]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*(X.size()[1]-1))<=torch.abs(0.02*self.w*self.w*(X.size()[1]-1)),x_mean_cons),self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [15]:
# Prover/ data owner side
theory_output = torch.sqrt(torch.var(data_tensor, correction = 1))
data_mean = torch.mean(data_tensor)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*(X.size()[1]-1))<=torch.abs(0.02*self.w*self.w*(X.size()[1]-1)),x_mean_cons),self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

theory output:  tensor(14.5800)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-40670,8],"logrows":16,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1520,"total_const_size":8,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [16]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 7.50620174407959 seconds
Theory output:  tensor(14.5800)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 14.5
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15846044116984004302, 17434658319873045084, 12685703199754313893, 2889686633444970017]]], 'proof': '0e104621edcd26304e287e2f55af8ae24b8662a4e0e9ab9977dc6d36ddd150a12a341f43af2c618c708e250c3b18a96b043f917f513f6fc722cee47d5bd7cb3d20c83f6e2cbb3a1f4d6d6367084c2d620a605117c66651d499c3471c4b9b7bab2b3bc2aa2b8d95d2ee1b06f484dfa59ec8f390aa3e087237c617b2c6badf62bf1ea89601dd825a46ab7f9aa7cc03507a24ca1411e34b809322893413dd1ea98a10d95fe89dd9e31644c7d20213c80930d46d4ee4f31c3e2f456f0022275b28291edcfc58424b5cc6e276dbcf905f61bb83ff724c3a7dc0b2f300a4578eb8958a162dcdaeda0c9a74a68a77a8477794e1d36952002b2d949f9d73d82a8065a85b268fc0c176ac020a9cd2f518f1340c70b35b224d9689d5db3dbaf7b7ef52add601c52d0294e6b5c38cc4a318750ad4cb0969be3f8c78a4c23724ccbd201b196b25064ef950b09f544d4b11c0d4c5da39d19d7e871ed8f07b5d693c4149f2

In [17]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15846044116984004302, 17434658319873045084, 12685703199754313893, 2889686633444970017]]]
proof boolean:  1.0
proof result 1 : 14.5
verified
